In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fpipe.pipeline.pipeline import run_pipeline
from fpipe.timestream import bandpass_cal, freq_rebin
from meerKAT_sim.plot import plot_waterfall

from tlpipe_fast.timestream import dispatch
from tlpipe_fast.timestream import rfi_flagging
from tlpipe_fast.timestream import sir_operate


In [4]:
DATA_BASE = '/scratch/users/ycli/fanalysis/'
DATA_Key = '0531'
if DATA_Key == '0531':
    file_temp = 'SDSS_N_2.5/20190531/SDSS_N_2.5_arcdrift%04d-%04d.h5'
    file_st = 3
    file_ed = 4 # max 22
    FILE_LIST = [file_temp%(i, i) for i in range(file_st, file_ed + 1)]
    #Tnoise_file = '/home/wangyg/data/ROACH_output/Tnosie_M_low.h5'

In [5]:
@run_pipeline
def analysis():    
    show = 0
    pipe_tasks  = []
    pipe_logging = 'info'
    data_base   = DATA_BASE
    pipe_outdir = data_base
    pipe_feedback = 0
    
    input_files = FILE_LIST
    
    freq_select = (200, 2248)
    #freq_select = (5000, 7048)
    
    #prefix = 'f300-f2348/bpcal_bpcal5_Tsys20/'
    prefix = 'raw/'
    
    pipe_tasks.append(bandpass_cal.Bandpass_Cal)

    bpcal_input_files  = ['%s/%s'%(prefix, f) for f in input_files]
    bpcal_output_files = ['bpcal_%d-%d/%s'%(freq_select + (f,)) for f in input_files]
    bpcal_noise_on_time = 1
    bpcal_pol_select = (0, 2)
    bpcal_freq_select = freq_select
    #bpcal_feed_select = [1,]
    bpcal_main_data = 'vis'
    bpcal_bad_freq_list = None #bad_freq_list # [[0, 50], [595, 605], [930, 955]] #bad_freq
    bpcal_bad_time_list = None #bad_time_list # [[0, 120], [3150, 3180], [3750, 3820]]
    bpcal_Tnoise_file = '/idia/users/ycli/fdata/fast_nd/ROACH_output/Tnosie_M_low.h5'
    bpcal_plot_spec = False

    return locals()

#analysis()

In [11]:

@run_pipeline
def rfi_flag():
    
    pipe_tasks  = []
    pipe_logging = 'info'
    data_base   = DATA_BASE
    pipe_outdir = data_base
    pipe_feedback = 0
    
    input_files = FILE_LIST
    
    #suffix = '_200-2248'
    suffix = '_5000-7048'
    
    #pipe_tasks.append(dispatch.Dispatch)
    #pipe_tasks.append(rfi_flagging.Flag)
    pipe_tasks.append(bandpass_cal.Normal_Tsys)
    
    ### parameters for Dispatch
    dp_input_files = [ data_base + 'bpcal%s/%s'%(suffix, f) for f in input_files]
    dp_start = 0
    dp_stop = None
    #dp_feed_select = [1, ]
    dp_extra_inttime = 0
    dp_exclude_bad = False
    dp_tag_input_iter = False
    dp_out = 'dp'
    
    
    ### parameters for Flag
    rf_in = dp_out
    #rf_input_files = ['bpcal_200-2248/%s'%f for f in input_files]
    rf_show_grogress = True
    rf_process_timing = True
    rf_first_threshold = 5.0
    # rf_distribution = 'Gaussian'
    rf_tk_size = 0.5
    rf_fk_size = 5.0
    # rf_flag_direction = ('time',)
    # rf_flag_direction = ('freq',)
    rf_output_files = [ 'sumthreshold_flagging%s/%s'%(suffix, f) for f in input_files ]
    rf_out = 'rf'
    rf_threshold_num = 2
    
    tsyscal_input_files = rf_output_files
    tsyscal_output_files = ['tsys_cal%s/%s'%(suffix, f) for f in input_files]
    tsyscal_T_sys = 20.
    tsyscal_timevars_poly = 16.
    #tsyscal_freq_select = (512, None)
    tsyscal_sub_mean = False
    
    return locals()

#rfi_flag()

INFO:tlpipe.pipeline.pipeline:Initializing task: <class 'fpipe.timestream.bandpass_cal.Normal_Tsys'>
INFO:tlpipe.pipeline.pipeline:Normal_Tsys reading data from files:
	/scratch/users/ycli/fanalysis/sumthreshold_flagging_5000-7048/SDSS_N_2.5/20190531/SDSS_N_2.5_arcdrift0003-0003.h5
	/scratch/users/ycli/fanalysis/sumthreshold_flagging_5000-7048/SDSS_N_2.5/20190531/SDSS_N_2.5_arcdrift0004-0004.h5


Reading parameters from dictionary.
Parameters set.
parameter: show_progress defaulted to value: False
parameter: process_timing defaulted to value: False
parameter: noise_on_time defaulted to value: 1
parameter: in defaulted to value: None
parameter: exclude defaulted to value: []
parameter: dist_axis defaulted to value: 0
parameter: freq_select defaulted to value: (0, None)
parameter: feed_select defaulted to value: (0, None)
parameter: iter_start defaulted to value: 0
parameter: tag_output_iter defaulted to value: True
parameter: show_info defaulted to value: False
parameter: chunk_shape defaulted to value: None
parameter: dry_run_time defaulted to value: 0
parameter: start defaulted to value: 0
parameter: iterable defaulted to value: False
parameter: out defaulted to value: None
parameter: check_status defaulted to value: True
parameter: output_failed_continue defaulted to value: False
parameter: iter_step defaulted to value: 1
parameter: time_select defaulted to value: (0, None)
p

INFO:tlpipe.pipeline.pipeline:Normal_Tsys writing data to files:
	/scratch/users/ycli/fanalysis/tsys_cal_5000-7048/SDSS_N_2.5/20190531/SDSS_N_2.5_arcdrift0003-0003.h5
	/scratch/users/ycli/fanalysis/tsys_cal_5000-7048/SDSS_N_2.5/20190531/SDSS_N_2.5_arcdrift0004-0004.h5




=                                        =
=        DONE FOR THE PIPELINE!!         =
=           CONGRATULATIONS!!            =
=                                        =
